In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb

%matplotlib inline

In [2]:
plt.rc('figure',dpi=100);

In [3]:
avisos = pd.read_csv("../../Data/fiuba_entrenamiento/gian/avisos.csv")
postulaciones = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulaciones.csv")
postulantes = pd.read_csv("../../Data/fiuba_entrenamiento/gian/postulantes.csv")

postulantes['sexo']=postulantes['sexo'].astype('category')
postulantes['Doctorado']=postulantes['Doctorado'].astype('category')
postulantes['Master']=postulantes['Master'].astype('category')
postulantes['Otro']=postulantes['Otro'].astype('category')
postulantes['Posgrado']=postulantes['Posgrado'].astype('category')
postulantes['Secundario']=postulantes['Secundario'].astype('category')
postulantes['Terciario/Técnico']=postulantes['Terciario/Técnico'].astype('category')
postulantes['Universitario']=postulantes['Universitario'].astype('category')

avisos['nombre_zona']=avisos['nombre_zona'].astype('category')
avisos['tipo_de_trabajo']=avisos['tipo_de_trabajo'].astype('category')
avisos['nivel_laboral']=avisos['nivel_laboral'].astype('category')
avisos['nombre_area']=avisos['nombre_area'].astype('category')

postulaciones['sepostulo']=1
postulaciones.drop(['fechapostulacion','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)

In [4]:
vistas = pd.read_csv("../../Data/fiuba_entrenamiento/gian/vistas.csv")
vistas.head()

,idAviso,timestamp,idpostulante,dia,mes,semana,diadelasemana,hora
0,1111780242,2018-02-23 18:38:13.187,YjVJQ6Z,23,2,8,Viernes,18
1,1112263876,2018-02-23 18:38:14.296,BmVpYoR,23,2,8,Viernes,18
2,1112327963,2018-02-23 18:38:14.329,wVkBzZd,23,2,8,Viernes,18
3,1112318643,2018-02-23 18:38:17.921,OqmP9pv,23,2,8,Viernes,18
4,1111903673,2018-02-23 18:38:18.973,DrpbXDP,23,2,8,Viernes,18


In [5]:
vistas.drop(['timestamp','dia','mes','semana','diadelasemana','hora'],axis=1,inplace=True)
vistas['cantidad']=0
vistas=vistas.groupby(['idAviso','idpostulante'],as_index=False).agg({'cantidad':'count'})
vistas=vistas.rename(columns={'idAviso': 'idaviso'})
vistas.head()

,idaviso,idpostulante,cantidad
0,18,BolNL,2
1,48375,RwVdKR,1
2,169730,1KjXB,2
3,169730,2AKzxa,2
4,169730,6LJ64,1


In [6]:
postulaciones2= pd.read_csv("../../Data/fiuba_entrenamiento/gian/no-postulaciones/1.csv")

postulaciones=pd.merge(postulaciones,postulantes,on='idpostulante')
postulaciones2=pd.merge(postulaciones2,postulantes,on='idpostulante')

postulaciones=pd.merge(postulaciones,avisos,on='idaviso')
postulaciones2=pd.merge(postulaciones2,avisos,on='idaviso')

postulaciones2.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,akjmXXq,0,FEM,28.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,MV6b6R9,0,FEM,30.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,zvxv1Rm,0,FEM,22.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,8oqJzL,0,MASC,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ez533B,0,MASC,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [7]:
postulaciones.head()

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,secundario,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion
0,1112257047,NM5M,1,FEM,47.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
1,1112257047,1kJqGb,1,FEM,34.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
2,1112257047,eOE9Rr,1,FEM,39.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False
3,1112257047,Zrx8Xz,1,FEM,39.0,-,-,-,-,-,...,True,False,False,True,False,False,False,False,False,False
4,1112257047,ZrKNQY,1,FEM,32.0,-,-,-,-,Graduado,...,True,False,False,True,False,False,False,False,False,False


In [8]:
postulaciones=postulaciones.append(postulaciones2, ignore_index=True)
print(len(postulaciones))
postulaciones.drop_duplicates(['idaviso','idpostulante'],keep='first',inplace=True)
print(len(postulaciones))

12377268
12332167


In [9]:
postulaciones=pd.merge(postulaciones,vistas,on=['idaviso','idpostulante'],how='left')
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
5318269,1112461848,EzEQWE0,1,MASC,33.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,3.0
5833640,1112318473,GNZQGw8,1,MASC,29.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,1.0
7766775,1112383388,LNP4YRQ,0,MASC,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,True,False,True,NaN
8682406,1112205105,2zQWZZY,0,FEM,24.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
5924620,1112453824,LNPox2j,1,MASC,21.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,2.0
12279670,1112328290,akDL49m,0,FEM,26.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
10614215,1112308404,vVwmrjQ,0,MASC,68.0,-,-,Graduado,-,Graduado,...,False,True,False,False,False,False,False,False,False,NaN
9342536,1112411732,NzrAJEx,0,FEM,22.0,-,-,Graduado,-,Graduado,...,False,False,False,False,False,False,False,False,False,NaN
1773772,1112309688,mzVVeKO,1,MASC,55.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,False,NaN
1559061,1112410555,Pm4lXGZ,1,FEM,32.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,False,1.0


In [10]:
postulaciones.cantidad=postulaciones.cantidad.fillna(0)
postulaciones.sample(20)

,idaviso,idpostulante,sepostulo,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
12248679,1112412376,xkPVL4a,0,FEM,25.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
11153970,1112416008,Ezez1J0,0,MASC,20.0,-,-,-,-,-,...,False,False,True,False,False,False,False,False,False,0.0
3146609,1112412238,RzrBbXo,1,MASC,26.0,-,-,-,-,Graduado,...,False,False,False,False,True,True,False,False,False,2.0
9276123,1112316313,JBmGmBk,0,MASC,31.0,-,-,Graduado,Graduado,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
11417231,1112411348,1QPLppR,0,MASC,21.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
3095391,1112418871,1jlbKN,1,FEM,30.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,1.0
10952346,1112421965,1Pejjb,0,MASC,40.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
6620082,1112345977,JBxkzYO,0,MASC,55.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,True,False,0.0
8897567,1112322826,6jXGQM,0,MASC,37.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
5902717,1112277833,96z4Eq5,1,MASC,23.0,-,-,-,-,-,...,False,False,False,False,False,True,False,False,False,0.0


In [11]:
train, test=train_test_split(postulaciones,test_size=0.10)
print("Train: ",len(train),"Test: ",len(test))
features=list(postulaciones.columns)
features.remove('idaviso')
features.remove('idpostulante')
features.remove('sepostulo')
features.remove('titulo')
features.remove('denominacion_empresa')

x_train=train[features]
y_train=train['sepostulo']

x_test=test[features]
y_test=test['sepostulo']
postulaciones=0
postulaciones2=0

Train:  11098950 Test:  1233217


In [14]:
import gc
gc.collect()

0

In [15]:
c=lgb.LGBMClassifier(learning_rate=0.01,objective='binary',num_leaves=4096,max_depth=13,n_estimators=250,colsample_bytree=0.8,n_jobs=-1,random_state=0,max_features=None)

dt=c.fit(x_train,y_train)
#c = joblib.load("../../Data/fiuba_entrenamiento/gian/modelos/....pkl") 

In [16]:
score=c.score(x_test,y_test)*100
print(score)

72.67626054457568


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [17]:
joblib.dump(c, "../../Data/fiuba_entrenamiento/gian/modelos/LGBM.pkl")

['../../Data/fiuba_entrenamiento/gian/modelos/LGBM.pkl']

In [18]:
prediccion = pd.read_csv("../../Data/fiuba_entrenamiento/test_final_100k.csv")
print(len(prediccion))
prediccion=pd.merge(prediccion,postulantes,on='idpostulante',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,avisos,on='idaviso',how='inner')
print(len(prediccion))
prediccion=pd.merge(prediccion,vistas,on=['idaviso','idpostulante'],how='left')
prediccion.cantidad=prediccion.cantidad.fillna(0)
prediccion.sample(10)

100000
100000
100000


,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,viajar,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad
67902,42291,1112423293,YWELWX,MASC,40.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,1.0
70941,32832,1112397590,ZRKoJ8,FEM,38.0,-,-,-,-,-,...,False,True,False,False,False,False,False,True,False,0.0
46768,2331,1111238486,E5axqN,MASC,46.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
6434,84550,1112466576,Rz634G9,FEM,24.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
8766,76228,1112464225,8MPdK3M,MASC,22.0,-,-,-,-,Graduado,...,False,True,False,False,False,False,False,False,False,0.0
35713,84262,1112466544,zv4jKx3,MASC,36.0,-,-,-,-,Graduado,...,False,True,False,False,True,False,False,False,False,1.0
19514,78959,1112464612,dYo0EWD,FEM,22.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,2.0
11069,25847,1112334806,8MrLJRL,FEM,21.0,-,-,-,-,Graduado,...,False,False,True,False,False,False,False,True,False,0.0
90471,57869,1112451926,a6GvZN,MASC,48.0,-,En Curso,-,-,Graduado,...,False,False,False,False,False,False,False,False,False,0.0
71839,17352,1112197441,6P2wAx,FEM,32.0,-,-,-,-,-,...,False,True,False,False,False,True,False,False,False,0.0


In [19]:
x_final=prediccion[features]
y_final=c.predict_proba(x_final)
y_final

array([[0.55287913, 0.44712087],
       [0.58212795, 0.41787205],
       [0.58130103, 0.41869897],
       ...,
       [0.84196037, 0.15803963],
       [0.57297411, 0.42702589],
       [0.90957099, 0.09042901]])

In [20]:
y_final2=[]
print(c.classes_)
print(len(y_final))
for elem in y_final:
    y_final2.append(elem[1])
print(len(y_final2))

[0 1]
100000
100000


In [21]:
prediccion['sepostulo']=y_final2

In [22]:
prediccion.head()

,id,idaviso,idpostulante,sexo,edad,Doctorado,Master,Otro,Posgrado,Secundario,...,empresa_importante,capacitacion,remuneracion_pretendida,graduados,estudiantes,conocimientos_especificos,buen_ambiente,discriminacion,cantidad,sepostulo
0,0,739260,6M9ZQR,FEM,42.0,Graduado,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.447121
1,1,739260,6v1xdL,MASC,30.0,-,-,Graduado,-,-,...,False,False,False,False,False,False,False,False,0.0,0.417872
2,2,739260,ezRKm9,FEM,36.0,-,-,-,-,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.418699
3,3,758580,1Q35ej,MASC,68.0,-,-,-,Graduado,Graduado,...,False,False,False,False,False,False,False,False,0.0,0.329989
4,4,758580,EAN4J6,FEM,32.0,-,-,-,-,-,...,False,False,False,False,False,False,False,False,0.0,0.369424


In [23]:
prediccion.drop(features,axis=1,inplace=True)
prediccion.drop(['idaviso','idpostulante','titulo','denominacion_empresa'],axis=1,inplace=True)
prediccion.head()

,id,sepostulo
0,0,0.447121
1,1,0.417872
2,2,0.418699
3,3,0.329989
4,4,0.369424


In [24]:
prediccion.to_csv("../../Data/fiuba_entrenamiento/gian/predicciones/prediccion14.csv",index=False)

In [25]:
prediccion.sepostulo.mean()

0.5890557084917467